In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import gdal
import pickle 
import datetime 
import multiprocessing
import cartopy
import xarray as xr
import os
import sys
#sys.path.append('/mnt/h1/projects/4DGreenland/scripts/lib/')
sys.path.append('/home/annpug/Desktop/annpug/scripts/lib.py')
import lib
import configparser
from osgeo import osr
import glob 
import geopandas as gpd
import seaborn as sns
from pyproj import Proj, Transformer

# Colors for plotting (might need to be removed once the backscatter profiles are removed.)
color_racmo =  (208/255,28/255,31/255)
color_mar = (225/255,126/255,4/255)
color_hirham = (47/255,158/255,49/255) 
colors,categories=lib.getColortableMeltConditions()
ct=((0,'#{:02x}{:02x}{:02x}'.format(*colors.GetColorEntry(0))),
    (1,'#{:02x}{:02x}{:02x}'.format(*colors.GetColorEntry(1))),
    (2,'#{:02x}{:02x}{:02x}'.format(*colors.GetColorEntry(2))),
    (3,'#{:02x}{:02x}{:02x}'.format(*colors.GetColorEntry(3))),
    (4,'#{:02x}{:02x}{:02x}'.format(*colors.GetColorEntry(4))))
labels = ['No melt','Increase of wet snow layer','Wet snow layer','Increase of refrozen layer','stationary']


# Correct Projection for the plots (EPSG: 3413, is specified wrongly in cartopy and is therefore manually defined)
plot_proj = cartopy.crs.Stereographic(central_longitude=-45,central_latitude=90,true_scale_latitude=70)

def world_to_pixel(geo_matrix, x1, y1,out_epsg):

    # tranform to polar sterographic:
    transformer = Transformer.from_crs('EPSG:4326',out_epsg)  
    x2,y2 = transformer.transform(y1,x1)

    # Get pixels:
    ul_x= geo_matrix[0]
    ul_y = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    pixel = int((x2 - ul_x) / x_dist)
    line = -int((ul_y - y2) / y_dist)
    return pixel, line

# Backscatter profiles  


In [ ]:
confpath = '/mnt/h1/projects/4DGreenland/scripts/4DGreenland_ASCAT_meltmap.ini'
config = configparser.ConfigParser()
config.read(confpath)

root_dir=config['PROJECT']['root_dir'].rstrip('/')
project=config['PROJECT']['project'].rstrip('/')
product_type='npy'
prj=config['PROJECT']['project']
sensor=config['DATA']['sensor']
timespan = f'{year}002_{year}362'
ipath=os.path.join(root_dir,project,product_type+'/')
opf = os.path.join(ipath,prj+'_'+sensor+'_'+timespan+'_data.npy')
metadict=lib.readYaml(os.path.join(ipath,prj+'_'+sensor+'_'+timespan+'_meta.yml'))
data = np.memmap(opf,dtype=np.dtype(metadict['dtype']), mode='r', shape=tuple(metadict['shape']))
filelist = np.load(ipath+prj+'_'+sensor+'_'+timespan+'_filelist.npy',allow_pickle=True)
mask = np.load(ipath+prj+'_'+sensor+'_'+timespan+'_mask.npy')
dates,vrtlist=list(zip(*filelist))

ds = gdal.Open('/mnt/h1/projects/4DGreenland/meltmap_v02/ASCAT/meltmap_v02_2015-01-15.tif')
geo_matrix = ds.GetGeoTransform()
proj = osr.SpatialReference(wkt=ds.GetProjection())
out_epsg='EPSG:'+str(proj.GetAttrValue('AUTHORITY',1))
ds=None

In [ ]:
st = 'NASA-SE'

if len(st) > 5:
    aws_measurements = pd.read_csv(f'/home/annpug/Desktop/annpug/AWS/combined/day/{st}_day.csv', header = 14, skipinitialspace=True, skiprows = range(15,25))
    aws_measurements = aws_measurements.rename(columns ={'# fields = timestamp': 'time', 'TA1':'t_u',
                                                        'Lat':'gps_lat', 'Lon':'gps_lon'})
    aws_measurements = aws_measurements.set_index('time')
    
# Load measurements from AWS
else: 
    aws_measurements = pd.read_csv(f'/home/annpug/Desktop/annpug/AWS/PROMICE/day/{st}_day.csv')
    aws_measurements = aws_measurements.set_index('time')

year_aws_measurements = aws_measurements.loc[f'{year}-01-01':f'{year}-12-31']
awsDates = pd.to_datetime(year_aws_measurements.index)# get station>
airTemp = year_aws_measurements.t_u

# Get longitud and latitude:
st_lat = year_aws_measurements.gps_lat[0]
st_lon = year_aws_measurements.gps_lon[0]

# Used to transfrom into polar stereographic:
#transformer = pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:3413')

x1,y1=world_to_pixel( geo_matrix, st_lon,st_lat,out_epsg) #'epsg:3413')
#x1=210
#y1=98

dates_class, year_ASCAT, doys,res = lib.classify(data[:,y1,x1],dates,config['PROCESSING'])
dates=np.asarray(dates)

point_mar = mar[:, y1,x1]
point_hirham = hirham[:,y1,x1]
point_racmo = racmo[:,y1,x1]
point_ens = ens_mean[:,y1,x1]
dates_rcm = pd.date_range(datetime.date(year,1, 1), periods = racmo.shape[0])

In [ ]:
ascat_season = [dates_rcm[int(onset_ascat[y1, x1])], dates_rcm[int(end_ascat[y1, x1])]]
hirham_season = [dates_rcm[int(start_hirham[y1, x1])], dates_rcm[int(end_hirham[y1, x1])]]
racmo_season = [dates_rcm[int(start_racmo[y1, x1])], dates_rcm[int(end_racmo[y1, x1])]]
mar_season = [dates_rcm[int(start_mar[y1, x1])], dates_rcm[int(end_mar[y1, x1])]]
ens_season = [dates_rcm[int(start_ens[y1, x1])], dates_rcm[int(end_ens[y1, x1])]]

plotTemperature = True
font= {'fontname':'Arial'}
labelfont = {'fontsize':14, **font}
tickfont = {'labelsize':10, **font}


fig, ax = plt.subplots(1,figsize=(18, 6))
for i,j in ct:
    z=0
    for k in dates_class[np.array(res==i)]:
        ax.axvspan(k,k +  np.timedelta64(24,'h'), facecolor=j, alpha=0.8,label=labels[i] if z == 0 else "")
        z+=1

ax.set_ylim([-15, 15])
#dateind=(dates>=datelims[0])&(dates<=datelims[1])
#ax.set_xlim(datelims)
#ax.plot(ASCAT_onset,0, '>', color = 'black', markersize=8)
#ax.plot(ASCAT_end,0, '>', color = 'black', markersize=8)
ax.plot(dates,data[:,y1,x1],'k-', label = 'ASCAT') #,label=f'lat {lat} lon {lon}')
ax.tick_params(labelsize=12)
ax.set_xlabel('date',**labelfont)
#fmt_month = mdates.MonthLocator()
#ax.xaxis.set_minor_locator(fmt_month)
ax.set_ylabel('Backscatter [dB]',**labelfont)

# ax[0].legend(loc="lower right")
if plotTemperature == True:
    axr = ax.twinx()
    #axr.plot(awsDates,airTemp,color='#1f78b4', label = 'PROMICE Ts')
    #racmo.melt.plot(ax=axr, color =color_racmo, label = 'RACMO', linewidth = 0.8)
    #mar.melt.plot(ax=axr, color = color_mar, label = 'MAR', linewidth = 0.8)
    plt.plot(dates_rcm, point_hirham, color =color_hirham, label = 'HIRHAM', linewidth = 0.8)
    plt.plot(dates_rcm, point_mar, color =color_mar, label = 'MAR', linewidth = 0.8)
    plt.plot(dates_rcm, point_racmo, color =color_racmo, label = 'RACMO', linewidth = 0.8)
    plt.plot(dates_rcm, point_ens, color ='purple', label = 'Ensemble mean', linewidth = 0.8)
    # Start and end of melt season in RCMs:
    #plt.plot(hirham_season, [0,0], '<', color = color_hirham )
    plt.plot(mar_season, [0,0], '<', color = color_mar)
    plt.plot(racmo_season, [0,0], '<', color = color_racmo)
    plt.plot(ens_season, [0,0], '<', color = 'purple')
    
    plt.plot(ascat_season, [0,0], '<', color = 'black')
    #plt.plot(dates_rcm[int(start_mar[y1, x1])],0, 'o',markersize=8, color = color_mar)
    #axr.plot([racmo_onset, racmo_end], [0,0], '<',markersize=8, color = color_racmo)
    axr.set_ylabel('Air Temperature [°C] or RCMs melt [mmWE/day]', **labelfont)
    axr.set_xlim(dates_rcm[0], dates_rcm[-1])
    axr.set_ylim([-15,10])
    #axr.set_title(f'Timeseries at point 1 {year}')
else: 
    ax.set_title(f'Timeseries at AWS {aws} {year}')

ax1_ylims = ax.axes.get_ylim()           # Find y-axis limits set by the plotter
ax1_yratio = float(ax1_ylims[0] / ax1_ylims[1])  # Calculate ratio of lowest limit to highest limit
ax2_ylims = axr.axes.get_ylim()           # Find y-axis limits set by the plotter
ax2_yratio = float(ax2_ylims[0] / ax2_ylims[1])  # Calculate ratio of lowest limit to highest limit
#ax.plot([dates[0],dates[-1]],[0,0],'k--') #,label=f'lat {lat} lon {lon}')

if ax1_yratio < ax2_yratio: 
    axr.set_ylim(bottom = ax2_ylims[1]*ax1_yratio)
else:
    ax.set_ylim(bottom = ax1_ylims[1]*ax2_yratio)
fig.legend(loc="upper left", bbox_to_anchor=(0,1), bbox_transform=ax.transAxes)
#fig.savefig(f'/home/annpug/Desktop/annpug/figures/point1_ASCAT_RCM_AWS_{year}.png')

# Points from QGIS

In [ ]:
#selected_points = gpd.read_file('/home/annpug/Desktop/annpug/QGIS/selected_points.shp')
selected_points = gpd.read_file('/home/annpug/Desktop/annpug/QGIS/meltseason_2018_selected_points.shp')

#selected_points = gpd.read_file(shapefile_name)
#selected_points = gpd.read_file('/home/annpug/Desktop/annpug/flowline_2_highres.shp')
#selected_points["inv_dist"] = max(selected_points['distance']) - selected_points['distance']
#selected_points = selected_points.sort_values('inv_dist')
#selected_points = selected_points.reset_index(drop=True)

for p in range(0, len(selected_points)):
    ul_x= geo_matrix[0]
    ul_y = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    x_pixel = int((selected_points.geometry.x[p] - ul_x) / x_dist)
    y_pixel = -int((ul_y - selected_points.geometry.y[p]) / y_dist)
    #print(x_pixel,y_pixel)
    #plt.scatter(x_pixel,y_pixel)
    
    point_mar = mar[:, y_pixel,x_pixel]
    point_hirham = hirham[:,y_pixel,x_pixel]
    point_racmo = racmo[:,y_pixel,x_pixel]
    point_ens = ens_mean[:,y_pixel,x_pixel]
    dates_class, year_ASCAT, doys,res = lib.classify(data[:,y_pixel,x_pixel],dates,config['PROCESSING'])
    dates=np.asarray(dates)
    point_ascat = data[:,y_pixel,x_pixel]

    fig, ax = plt.subplots(1,figsize=(10, 3))
    
    for i,j in ct:
        z=0
        for k in dates_class[np.array(res==i)]:
            ax.axvspan(k,k +  np.timedelta64(24,'h'), facecolor=j, alpha=0.8)
            z+=1
    ax.plot(dates,point_ascat,'k-', label = f'POINT {p}', linewidth = 0.8) #,label=f'lat {lat} lon {lon}')
    #ax.plot(doys,res)
    ax.tick_params(labelsize=10)
    ax.set_xlabel('date')
    ax.set_ylabel('Backscatter [dB]')
    ax.set_ylim([min(point_ascat)-3, 15])
    #ax.set_ylim([-26, 15])
    #ax.set_title(f'Point {p+1}')
    axr = ax.twinx()
    #ascat_season = [dates_rcm[int(onset_ascat[y_pixel, x_pixel])], dates_rcm[int(end_ascat[y_pixel, x_pixel])]]
    
    try: 
        hirham_season = [dates_rcm[int(start_hirham[y_pixel, x_pixel])], dates_rcm[int(end_hirham[y_pixel, x_pixel])]]
        axr.plot(hirham_season, [0,0], '<', color = color_hirham)
    except ValueError:
        print(f'No start/end for Hirham for point {p+1}')
    
    try: 
        racmo_season = [dates_rcm[int(start_racmo[y_pixel, x_pixel])], dates_rcm[int(end_racmo[y_pixel, x_pixel])]]
        axr.plot(racmo_season, [0,0], '<', color = color_racmo)
    except ValueError:
        print(f'No start/end for Racmo for point {p+1}')
    
    try:  
        mar_season = [dates_rcm[int(start_mar[y_pixel, x_pixel])], dates_rcm[int(end_mar[y_pixel, x_pixel])]]
        axr.plot(mar_season, [0,0], '<', color = color_mar)     
    except ValueError:
        print(f'No start/end for Mar for point {p+1}')
    try: 
        ens_season = [dates_rcm[int(start_ens[y_pixel, x_pixel])], dates_rcm[int(end_ens[y_pixel, x_pixel])]]
        axr.plot(ens_season, [0,0], '<', color = 'purple')
    except ValueError:
        print(f'No start/end for Ensemble mean for point {p}')
    
        
    axr.plot(dates_rcm, point_hirham, color =color_hirham, label = 'HIRHAM', linewidth = 0.8)
    axr.plot(dates_rcm, point_mar, color =color_mar, label = 'MAR', linewidth = 0.8)
    axr.plot(dates_rcm, point_racmo, color =color_racmo, label = 'RACMO', linewidth = 0.8)
    axr.plot(dates_rcm, point_ens, color ='purple', label = 'Ensemble mean', linewidth = 0.8)
    #axr.plot(dates_ascat, ascat[:, y_pixel, x_pixel])
    #plt.plot(ascat_season, [0,0], '<', color = 'black')

    axr.set_ylabel('RCMs melt [mmWE/day]')
    axr.set_xlim(dates[0], dates[-1])
    #axr.set_ylim(-60, 60)


    # limit stuff:
    ax1_ylims = ax.axes.get_ylim()           # Find y-axis limits set by the plotter
    ax1_yratio = float(ax1_ylims[0] / ax1_ylims[1])  # Calculate ratio of lowest limit to highest limit
    ax2_ylims = axr.axes.get_ylim()           # Find y-axis limits set by the plotter
    ax2_yratio = float(ax2_ylims[0] / ax2_ylims[1])  # Calculate ratio of lowest limit to highest limit
    #ax.plot([dates[0],dates[-1]],[0,0],'k--') #,label=f'lat {lat} lon {lon}')

    if ax1_yratio < ax2_yratio: 
        axr.set_ylim(bottom = ax2_ylims[1]*ax1_yratio)
    else:
        ax.set_ylim(bottom = ax1_ylims[1]*ax2_yratio)
    ax.legend(loc = 'upper left')
    fig.savefig(f'/home/annpug/Desktop/annpug/figures/{year}_profile_at_point_{p}.png')

In [ ]:
# PLOT OF point location
fig, axs = plt.subplots(1, 3, figsize = (14,5.5), subplot_kw={'projection': cartopy.crs.NorthPolarStereo(central_longitude=-45)})

#ax[0].imshow(onset_dif_ens, vmin = -30,vmax = 30, cmap = 'RdBu' )
#ax[1].imshow(end_dif_ens, vmin = -30,vmax = 30, cmap = 'RdBu')

#im = ax[2].imshow(duration_dif_ens, vmin = -30,vmax = 30, cmap = 'RdBu' )
#plt.colorbar(im , ax = ax[0], label = 'Duration of melt season [#Days]')
fig.suptitle('ASCAT - ensenble mean', fontweight = 'bold')


plot_greenland(onset_dif_ens, axs[0], crs_epsg, cbar = False,
               vmin = minlim, vmax = maxlim, cmap = cmap)
plot_greenland(end_dif_ens, axs[1], crs_epsg, cbar = False,
               vmin = minlim, vmax = maxlim, cmap = cmap)
plot_greenland(duration_dif_ens, axs[2], crs_epsg, cbar = True,  
               cbar_label = r'$\Delta$days', 
               vmin = minlim, vmax = maxlim, cmap = cmap)


for p in range(0, len(selected_points)):
    
    #ascat_season = [dates_rcm[int(onset_ascat[y_pixel, x_pixel])], dates_rcm[int(end_ascat[y_pixel, x_pixel])]]
    #hirham_season = [dates_rcm[int(start_hirham[y_pixel, x_pixel])], dates_rcm[int(end_hirham[y_pixel, x_pixel])]]
    #racmo_season = [dates_rcm[int(start_racmo[y_pixel, x_pixel])], dates_rcm[int(end_racmo[y_pixel, x_pixel])]]
    #mar_season = [dates_rcm[int(start_mar[y_pixel, x_pixel])], dates_rcm[int(end_mar[y_pixel, x_pixel])]]
    #ens_season = [dates_rcm[int(start_ens[y_pixel, x_pixel])], dates_rcm[int(end_ens[y_pixel, x_pixel])]]
    axs[0].scatter(selected_points.geometry.x[p], selected_points.geometry.y[p],facecolors='none', edgecolors='black')
    axs[0].annotate(f'{p+1}', (selected_points.geometry.x[p]+3e4, selected_points.geometry.y[p]), color = 'black', fontweight = 'bold')
    axs[1].scatter(selected_points.geometry.x[p], selected_points.geometry.y[p], facecolors='none', edgecolors='grey')
    axs[1].annotate(f'{p+1}', (selected_points.geometry.x[p]+3e4, selected_points.geometry.y[p]), color = 'grey', fontweight = 'bold')
    axs[2].scatter(selected_points.geometry.x[p], selected_points.geometry.y[p], facecolors='none', edgecolors='lightgrey')
    axs[2].annotate(f'{p+1}', (selected_points.geometry.x[p]+3e4, selected_points.geometry.y[p]), color = 'grey', fontweight = 'bold')
    
#fig.tight_layout(pad=-0)

# Backscatter along flowlines

In [ ]:
import matplotlib

fl_no = 3
shapefile_name = f'/home/annpug/Desktop/annpug/QGIS/flowline_{fl_no}_highres.shp'
selected_points = gpd.read_file(shapefile_name)
selected_points["inv_dist"] = max(selected_points['distance']) - selected_points['distance']
selected_points = selected_points.sort_values('inv_dist')
selected_points = selected_points.reset_index(drop=True)

# Get pixel index: 
ul_x= geo_matrix[0]
ul_y = geo_matrix[3]
x_dist = geo_matrix[1]
y_dist = geo_matrix[5]
x_pixel = ((selected_points.geometry.x - ul_x) / x_dist).astype(int)
y_pixel = -((ul_y - selected_points.geometry.y) / y_dist).astype(int)
selected_points['x_pixel'] = x_pixel 
selected_points['y_pixel'] = y_pixel 

#selected_points = selected_points.reset_index()
#selected_points.to_file(shapefile_name)


cmap=matplotlib.cm.viridis
colors = cmap(np.linspace(0,1,len(selected_points)))

fig, ax = plt.subplots(1,figsize=(14, 5))

#fig2, ax2 = plt.subplots(1,figsize=(4, 5.5),subplot_kw={'projection': cartopy.crs.NorthPolarStereo(central_longitude=-45)}) 
#plot_greenland(onset_dif_ens,ax2, crs_epsg, cbar = False,
#               vmin = minlim, vmax = maxlim, cmap = cmap)

for p in range(0, len(selected_points)):
    point_ascat = data[:,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ax.plot(dates,point_ascat,'-', label = f'POINT {p+1}', linewidth = 0.8, color = colors[p]) #,label=f'lat {lat} lon {lon}')
    
    #ax2.scatter(selected_points.geometry.x[p], selected_points.geometry.y[p], facecolors='none', edgecolors='black', s = 10)
    #ax2.annotate(f'{p+1}', (selected_points.geometry.x[p]+3e4, selected_points.geometry.y[p]), color = 'black', fontweight = 'bold')
    
norm = plt.Normalize(0, max(selected_points['distance']/1000))
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    
sm.set_array([])
ax.figure.colorbar(sm, ax=ax, label = 'Distance [km]')  
ax.set_xlim(dates[0], dates[-1])
ax.set_ylabel('Backscatter [dB]')
ax.set_title(f'Backscatter profiles along flowline {fl_no}')
#ax2.set([200,0 , 200, 100])
#ax2.set_extent([-600000 ,-500000, -60000, -1400000],crs = cartopy.crs.NorthPolarStereo(central_longitude=0))



In [ ]:
ascat_along_flowline= np.zeros((len(selected_points),10))
ascat_class = np.zeros((len(selected_points),10))
start_time = 90
dt = 3
for p in range(0, len(selected_points)):
    ascat_along_flowline[p,0] = data[start_time,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,1] = data[start_time+dt,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,2] = data[start_time+dt*2,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,3] = data[start_time+dt*3,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,4] = data[start_time+dt*4,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,5] = data[start_time+dt*5,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,6] = data[start_time+dt*6,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,7] = data[start_time+dt*7,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,8] = data[start_time+dt*8,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    ascat_along_flowline[p,9] = data[start_time+dt*9,selected_points.y_pixel[p],selected_points.x_pixel[p]]
    
    ascat_class[p, :] = np.array([ascat[dates[start_time].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]], 
                         ascat[dates[start_time+dt].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*2].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*3].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*4].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*5].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*6].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*7].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*8].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ascat[dates[start_time+dt*9].timetuple().tm_yday, selected_points.y_pixel[p],selected_points.x_pixel[p]],
                         ]).astype(int)
    
    
    
dates4label = [dates[start_time], dates[start_time+dt], dates[start_time+dt*2],
               dates[start_time+dt*3], dates[start_time+dt*4], dates[start_time+dt*5], 
               dates[start_time+dt*6], dates[start_time+dt*7], dates[start_time+dt*8], dates[start_time+dt*9]]
dates4label = [d.date() for d in dates4label]

fig, ax = plt.subplots(1,figsize=(14, 5))
ax.plot(selected_points.inv_dist/1000, ascat_along_flowline,'-o',label = dates4label)#, label = f'POINT {p+1}', linewidth = 0.8, color = colors[p]) #,label=f'lat {lat} lon {lon}')
#ax.scatter(selected_points.inv_dist/1000, ascat_along_flowline)#, label = f'POINT {p+1}', linewidth = 0.8, color = colors[p]) #,label=f'lat {lat} lon {lon}')
ax.set_xlim(0,selected_points.inv_dist[p]/1000 )
ax.set_xlabel('Disatnce from outlet [km]')
ax.set_ylabel('Backscatter [dB]')
ax.set_title(f'Backscatter along flowline {fl_no}')
ax.legend()
plt.show()